In [1]:
import pandas as pd

from netam.common import parameter_count_of_model
from shmex import shm_zoo

Using Metal Performance Shaders


In [2]:
def attr_with_default(obj, attr1, attr2, default="-"):
    return getattr(getattr(obj, attr1), attr2) if hasattr(obj, attr1) else default

model_summary_rows = []

model_names = ['fivemer', 'rsshmoof', 'cnn_joi_lrg', 'cnn_ind_med', 'cnn_ind_lrg']

for model_name in model_names:
    model = shm_zoo.create_model(model_name)
    parameter_count = parameter_count_of_model(model)
    embedding_dim = attr_with_default(model, "conv", "in_channels")
    filter_count = attr_with_default(model, "conv", "out_channels")
    dropout = attr_with_default(model, "dropout", "p")
    kernel_size = attr_with_default(model, "conv", "kernel_size")[0]
    long_name = shm_zoo.long_name_of_short_name(model_name)
    model_summary_rows.append((model_name, long_name, kernel_size, embedding_dim, filter_count, dropout, parameter_count))

summary_df = pd.DataFrame(model_summary_rows, columns=["model", "model name", "kernel_size", "embedding_dim", "filter_count", "dropout", "parameter_count"])
# apply shm_zoo.fix_parameter_count to summary_df
summary_df["parameter_count"] = summary_df.apply(shm_zoo.fix_parameter_count, axis=1)
summary_df = summary_df.sort_values("parameter_count")
summary_df.to_csv("output/model_parameter_count.csv", index=False)
summary_df

,model,model name,kernel_size,embedding_dim,filter_count,dropout,parameter_count
2,cnn_joi_lrg,CNN Joined Large,11,7,19,0.3,2057
0,fivemer,5mer,-,-,-,-,3077
1,rsshmoof,Spisak,-,-,-,-,3576
3,cnn_ind_med,CNN Indep Medium,9,7,16,0.2,4539
4,cnn_ind_lrg,CNN Indep Large,11,7,19,0.3,5931


In [3]:
df = summary_df
df = df.drop(columns=["model"])
rename_dict = {
    "kernel_size": "Kernel size",
    "embedding_dim": "Embedding dim",
    "filter_count": "Filter count",
    "parameter_count": "Parameters"
}
# rename columns
df = df.rename(columns=rename_dict)
# replace the Parameters column with a string version with commas
df["Parameters"] = df["Parameters"].apply(lambda x: f"{x:,}")
df.to_latex("output/model_parameter_count.tex", index=False, escape=False)